<a href="https://colab.research.google.com/github/Hegdedattatraya/EdunetInternship/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

# Load dataset from Drive (update path!)
# The file appears to be a CSV, not an XLS, despite the extension.
xls_path = "/content/drive/My Drive/juliet_dataset.xls"

try:
    # Attempt to read as CSV
    df = pd.read_csv(xls_path)
    print(f"✅ Dataset loaded successfully as CSV: {df.shape[0]} samples")
except Exception as e:
    print(f"❌ Failed to load dataset as CSV: {e}")
    # If it's confirmed to be an Excel file but recent format (.xlsx),
    # you might need to use the 'openpyxl' engine and check the file extension.
    # If it truly is an old .xls file and xlrd is giving issues,
    # you might need to inspect the file's contents manually to understand its format.
    # If it was an .xlsx file incorrectly named as .xls:
    # try:
    #     df = pd.read_excel(xls_path, engine="openpyxl")
    #     print(f"✅ Dataset loaded successfully as XLSX with openpyxl: {df.shape[0]} samples")
    # except Exception as e_xlsx:
    #     print(f"❌ Failed to load dataset as XLSX: {e_xlsx}")
    pass # Handle other cases or re-raise

✅ Dataset loaded successfully as CSV: 34403 samples


In [3]:
csv_path = "/content/drive/My Drive/juliet_dataset.csv"
df.to_csv(csv_path, index=False)

print("✅ Conversion successful! Saved as juliet_dataset.csv")

✅ Conversion successful! Saved as juliet_dataset.csv


In [ ]:
from transformers import RobertaTokenizer

# Load CodeBERT tokenizer
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")

# Tokenize source code
df["tokenized_code"] = df["code"].apply(lambda x: tokenizer.encode(x, truncation=True, padding="max_length", max_length=128))

# Save tokenized dataset
df.to_csv("tokenized_vulnerability_dataset.csv", index=False)
print("✅ Tokenization successful!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

In [ ]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import RobertaForSequenceClassification
from torch.optim import AdamW  # ✅ Correct

# Convert data to tensors
X = torch.tensor(df["tokenized_code"].tolist(), dtype=torch.long)
y = torch.tensor(df["label"].tolist(), dtype=torch.long)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Load CodeBERT model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RobertaForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels=2)
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

In [6]:
from torch.cuda.amp import autocast, GradScaler

scaler = GradScaler()
model.train()

for epoch in range(3):  # Adjust epochs as needed
    for batch in train_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        with autocast():
            outputs = model(inputs).logits
            loss = loss_fn(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    print(f"Epoch {epoch+1}: Loss = {loss.item()}")

# Save trained model to Google Drive
model.save_pretrained("/content/drive/My Drive/codebert_vulnerability_model")
print("✅ Model training complete!")

<ipython-input-6-53f05b6da0c9>:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-6-53f05b6da0c9>:12: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Epoch 1: Loss = 8.273124694824219e-05
Epoch 2: Loss = 4.649162292480469e-06
Epoch 3: Loss = 1.5497207641601562e-06
✅ Model training complete!


In [7]:
model.eval()
correct = 0
total = 0

for batch in test_loader:
    inputs, labels = batch
    inputs, labels = inputs.to(device), labels.to(device)

    outputs = model(inputs).logits
    predictions = outputs.argmax(dim=1)

    correct += (predictions == labels).sum().item()
    total += labels.size(0)

accuracy = correct / total
print(f"✅ Model Accuracy: {accuracy * 100:.2f}%")

✅ Model Accuracy: 99.99%


In [10]:
def predict_code_vulnerability(code_snippet):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Tokenize input code
    tokenized = tokenizer.encode(code_snippet, truncation=True, padding="max_length", max_length=128, return_tensors="pt").to(device)

    # Get model prediction
    outputs = model(tokenized).logits
    prediction = outputs.argmax(dim=1).item()

    # Classification result
    result = "Vulnerable Code 🚨" if prediction == 1 else "Secure Code ✅"
    return result

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model & tokenizer
model_path = "/content/drive/My Drive/codebert_vulnerability_model"
model = RobertaForSequenceClassification.from_pretrained(model_path)
model.to(device)
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")

print("✅ Model and tokenizer loaded successfully!")

✅ Model and tokenizer loaded successfully!


In [12]:
code_snippet = "int user_input = get_input();"
tokenized = tokenizer.encode(code_snippet, truncation=True, padding="max_length", max_length=128, return_tensors="pt").to(device)

print("✅ Tokenized input:", tokenized)

✅ Tokenized input: tensor([[    0,  2544,  3018,  1215, 46797,  5457,   120,  1215, 46797, 47006,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,    

In [13]:
def predict_code_vulnerability(code_snippet):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Tokenization
    tokenized = tokenizer.encode(code_snippet, truncation=True, padding="max_length", max_length=128, return_tensors="pt").to(device)
    print("✅ Tokenized Input:", tokenized)

    # Get model prediction
    outputs = model(tokenized).logits
    print("✅ Model Outputs:", outputs)

    prediction = outputs.argmax(dim=1).item()
    print("✅ Predicted Class:", prediction)

    # Classification result
    result = "Vulnerable Code 🚨" if prediction == 1 else "Secure Code ✅"
    return result

In [14]:
print(predict_code_vulnerability("int user_input = get_input();"))

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


✅ Tokenized Input: tensor([[    0,  2544,  3018,  1215, 46797,  5457,   120,  1215, 46797, 47006,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,    

In [15]:
from google.colab import drive
drive.mount('/content/drive')

# Save trained model
model.save_pretrained("/content/drive/My Drive/codebert_vulnerability_model")
print("✅ Model saved to Google Drive!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model saved to Google Drive!
